# 2DG Perturbation Rep 1 Extracting Topological Skeletons
In this notebook we are extracting topological skeletons in order to measure the distance along the medial axis. First we load the data and define a function so that we can process all of the images at the same time (making them the same size so we can stack them in numpy)

In [1]:
import os 
import skimage.io as io
import pyclesperanto_prototype as cle
import napari
import organoid_prediction_python as opp
import numpy as np

def make_images_equal_again(image_list):
    max_x = np.max([img.shape[1] for img in image_list])
    max_y = np.max([img.shape[0] for img in image_list])
    
    out_image_list = []
    
    for img in image_list:
        shape = img.shape
        if shape[0] < max_y:
            y_difference = max_y-shape[0]
            black_strip = np.full((y_difference, shape[1]),0)
            img = np.concatenate([black_strip,img],axis=0)
        if shape[1] < max_x:
            x_diff = max_x - shape[1]
            black_strip = np.full((max_y, x_diff),0)
            img = np.concatenate([black_strip,img],axis=1)
        out_image_list.append(img)
        
    return np.array(out_image_list)

home_directory = r"C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/"
path = home_directory + "image_data/Processed Data/Perturbation Analysis/Mean Projections/"

file_list = [file for file in os.listdir(path) if file.endswith("crop.tif")]
file_list[:5]

['2DG_2mM_72h_P1_17_001_crop.tif',
 '2DG_2mM_72h_P1_18_001_crop.tif',
 '2DG_2mM_72h_P1_19_001_crop.tif',
 '2DG_2mM_72h_P1_20_001_crop.tif',
 '2DG_2mM_72h_P1_21_001_crop.tif']

In [2]:
collection = io.imread_collection(path+"/*crop.tif")
collection.files[:5]

['C:\\Users\\savill\\OneDrive\\Documents\\PhD Jesse\\Embryonic_organoid_prediction\\Processing dataset for Paper\\TLS_2D_morphostate_investigation/image_data/Processed Data/Perturbation Analysis/Mean Projections\\2DG_2mM_72h_P1_17_001_crop.tif',
 'C:\\Users\\savill\\OneDrive\\Documents\\PhD Jesse\\Embryonic_organoid_prediction\\Processing dataset for Paper\\TLS_2D_morphostate_investigation/image_data/Processed Data/Perturbation Analysis/Mean Projections\\2DG_2mM_72h_P1_18_001_crop.tif',
 'C:\\Users\\savill\\OneDrive\\Documents\\PhD Jesse\\Embryonic_organoid_prediction\\Processing dataset for Paper\\TLS_2D_morphostate_investigation/image_data/Processed Data/Perturbation Analysis/Mean Projections\\2DG_2mM_72h_P1_19_001_crop.tif',
 'C:\\Users\\savill\\OneDrive\\Documents\\PhD Jesse\\Embryonic_organoid_prediction\\Processing dataset for Paper\\TLS_2D_morphostate_investigation/image_data/Processed Data/Perturbation Analysis/Mean Projections\\2DG_2mM_72h_P1_20_001_crop.tif',
 'C:\\Users\\sav

Taking a Peek

In [3]:
viewer = napari.Viewer()
viewer.add_image(collection[22])

<Image layer 'Image' at 0x24f8737dee0>

Binarizing the images:

In [4]:
import napari_simpleitk_image_processing as nsitk  # version 0.4.5

def processing(image):
    image1_gb = cle.gaussian_blur(image, None, 10.0, 10.0, 0.0)
    image2_to = cle.threshold_otsu(image1_gb)
    image3_B = nsitk.binary_fill_holes(image2_to)

    return image3_B

In [5]:
results = []
for image in collection:
    results.append(processing(image))

len(results)

320

Taking a look

In [6]:
viewer.add_image(make_images_equal_again(collection))
viewer.add_labels(make_images_equal_again(results).astype(int))

<Labels layer 'Labels' at 0x24fb12d8a90>

Make surte we don't have multiple masks and then performing skeletonization:

In [8]:
only_one_object = opp.keep_labels_closest_to_stack_median(results)

In [29]:

import napari_segment_blobs_and_things_with_membranes as nsbatwm  # version 0.3.4
skeletons = [nsbatwm.skeletonize(label) for label in only_one_object]


Checking the Results and Saving

In [31]:
viewer.add_image(make_images_equal_again(collection))
viewer.add_labels(make_images_equal_again(skeletons).astype(int))

<Labels layer 'Labels [1]' at 0x24fcaaeaf10>

Finding layer failed. Change was not stored


(Make sure 'QVector<int>' is registered using qRegisterMetaType().)
(Make sure 'QVector<int>' is registered using qRegisterMetaType().)
(Make sure 'QVector<int>' is registered using qRegisterMetaType().)
(Make sure 'QVector<int>' is registered using qRegisterMetaType().)


In [33]:
home_directory = r"C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/"
out_folder = home_directory + "image_data/Processed Data/Perturbation Analysis/Skeletons/"

if not(os.path.isdir(out_folder)):
    os.mkdir(out_folder)

for image,name in zip(skeletons,collection.files):
    file_name = os.path.split(name)[1]
    io.imsave(out_folder+file_name[:-4] + "_skeleton.tif",image)

(Parent is QtLabelsControls(0x25153e74110), parent's thread is QThread(0x24f8b5a3310), current thread is QThreadPoolThread(0x24f9574d0d0)
(Parent is QtLabelsControls(0x25153e74110), parent's thread is QThread(0x24f8b5a3310), current thread is QThreadPoolThread(0x24f9574d0d0)
(Parent is QtLabelsControls(0x25153e74110), parent's thread is QThread(0x24f8b5a3310), current thread is QThreadPoolThread(0x24f9574d0d0)
(Parent is QtLabelsControls(0x25153e74110), parent's thread is QThread(0x24f8b5a3310), current thread is QThreadPoolThread(0x24f9574d0d0)
